In [4]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING  --disable_caching ip_Scene"
video2_scene = " -v WARNING  --disable_caching ip2_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s ip_Scene"

In [5]:
class InclinedPlane(Mobject):
    def __init__(self, starting_center, height, width, ejection_time = 0, g = 9.81, **kwargs):
        super().__init__(**kwargs)

        self.ax_center = starting_center            # this is good for exaxtly nothing because manim doesn't put ax origin into center [desgracia]
        self.wedge_height = height
        self.wedge_width = width
        self.ejection_time = ejection_time
        self.g = g
        self.angle = np.arctan(height / width)


        # coordinate system c
        self.ax_c = Axes(x_range = (-1, self.wedge_width+1, 1), y_range = (-self.wedge_height, self.wedge_height, 1), x_length = self.wedge_width+2, y_length = 2*self.wedge_height,
            axis_config = {'color': BLACK, "tip_width": 0.15, "tip_height": 0.15}
        ).move_to(self.ax_center)
        self.ax_c_xlabel = self.ax_c.get_x_axis_label(Tex(r"$x$", font_size = 36, color = BLACK)).shift(0.5 * DOWN)
        self.ax_c_ylabel = self.ax_c.get_y_axis_label(Tex(r"$y$", font_size = 36, color = BLACK)).shift(0.5 * LEFT)


        # coordinate system c'
        self.ax_c_star = Axes(x_range = (-1, self.wedge_width+1, 1), y_range = (-self.wedge_height, self.wedge_height, 1), x_length = self.wedge_width+2, y_length = 2*self.wedge_height,
            axis_config = {'color': BLACK, "tip_width": 0.15, "tip_height": 0.15}
        ).move_to(self.ax_center).rotate(about_point = self.ax_c.c2p(0, 0, 0), angle = -self.angle).set_opacity(0.125)
        self.ax_c_star_xlabel = self.ax_c.get_x_axis_label(Tex(r"$x'$", font_size = 36, color = BLACK)).rotate(about_point = self.ax_c.c2p(0, 0, 0), angle = -self.angle).set_opacity(0.125)
        self.ax_c_star_ylabel = self.ax_c.get_y_axis_label(Tex(r"$y'$", font_size = 36, color = BLACK)).rotate(about_point = self.ax_c.c2p(0, 0, 0), angle = -self.angle).set_opacity(0.125)

        # inclined plane
        self.inclined_plane = Line(start = self.ax_c.c2p(0, 0, 0), end = self.ax_c.c2p(self.wedge_width, -self.wedge_height, 0), color = BLACK, stroke_opacity = 0.5)
        self.inclined_plane_fill = Polygon(self.ax_c.c2p(0, 0, 0), self.ax_c.c2p(self.wedge_width, -self.wedge_height, 0), self.ax_c.c2p(0, -self.wedge_height, 0), fill_color = BLACK, fill_opacity = 0.125, stroke_opacity = 0)
        self.add(self.ax_c)
        self.add(self.inclined_plane, self.inclined_plane_fill)


    # waggon rolling down the plane
    def get_waggon(self, t):
        waggon_x_coord = self.g/2*(t)**2 * np.sin(self.angle) * np.cos(self.angle)
        waggon_y_coord = -self.g/2*(t)**2 * np.sin(self.angle) * np.sin(self.angle)
        waggon_pos_increment = 0.25/2*(np.cos(self.angle)*UP + np.sin(self.angle)*RIGHT)
        waggon_pos = self.ax_c.c2p(self.g/2*t**2 * np.sin(self.angle) * np.cos(self.angle), -self.g/2*t**2 * np.sin(self.angle) * np.sin(self.angle), 0) + 0.25/2*(np.cos(self.angle)*UP + np.sin(self.angle)*RIGHT)
        # waggon = Rectangle(height = 0.25, width = 0.5, color = BLACK, fill_color = BLACK, fill_opacity = 0.5
        #     ).move_to(waggon_pos).rotate(about_point = waggon_pos, angle = -self.angle)
            
        top_left = self.ax_c.c2p(waggon_x_coord-0.5/2, waggon_y_coord+0.25/2, 0)
        top_right = self.ax_c.c2p(waggon_x_coord+0.5/2, waggon_y_coord+0.25/2, 0)
        down_right = self.ax_c.c2p(waggon_x_coord+0.5/2, waggon_y_coord-0.25/2, 0)
        down_left = self.ax_c.c2p(waggon_x_coord-0.5/2, waggon_y_coord-0.25/2, 0)
        waggon = Polygon(top_left, top_right, down_right, down_left, color = BLACK, fill_color = BLACK, fill_opacity = 0.5).rotate(about_point = waggon_pos, angle = -self.angle).shift(waggon_pos_increment)
        waggon.position = waggon_pos

        # waggon vectors
        waggon.gravitation = Line(start = waggon_pos, end = waggon_pos + 1.5*DOWN, color = BLACK).add_tip(tip_length = 0.15, tip_width = 0.15)
        waggon.gravitation_descriptor = Tex(r"$\Vec{g}$", font_size = 32, color = BLACK).next_to(waggon.gravitation, RIGHT).shift(0.15*LEFT)
        waggon.perp = Line(start = waggon_pos, end = waggon_pos + 1.5*np.cos(self.angle)*(np.cos(self.angle)*DOWN - np.sin(self.angle)*RIGHT), color = BLACK).add_tip(tip_length = 0.15, tip_width = 0.15).set_opacity(0.5)
        waggon.perp_descriptor = Tex(r"$\Vec{g}_\mathrm{y'}$", font_size = 32, color = BLACK).next_to(waggon.perp, LEFT).shift(0.3*RIGHT).set_opacity(0.5)
        waggon.line = Line(start = waggon_pos, end = waggon_pos + 1.5*np.sin(self.angle)*(np.sin(self.angle)*DOWN + np.cos(self.angle)*RIGHT), color = BLACK).add_tip(tip_length = 0.15, tip_width = 0.15).set_opacity(0.5)
        waggon.line_descriptor = Tex(r"$\Vec{g}_\mathrm{x'}$", font_size = 32, color = BLACK).next_to(waggon.line, RIGHT).shift(0.1*LEFT).set_opacity(0.5)
        return waggon
    

    # ball getting ejected
    def get_ball(self, t):
        v0 = 4.5 * self.g / 10                                      # starting velocity
        # coordinate of the waggon
        waggon_c = np.array([self.g/2*t**2 * np.sin(self.angle) * np.cos(self.angle), -self.g/2*t**2 * np.sin(self.angle) * np.sin(self.angle), 0])
        # additional ejection position change
        ball_increment_c = np.array([v0 * (t-self.ejection_time) * np.sin(self.angle), -self.g/2*(t-self.ejection_time)**2 + v0 * (t-self.ejection_time) * np.cos(self.angle), 0])
        # speed and position of the waggon at ejection
        waggon_eject_pos = np.array([self.g/2*self.ejection_time**2 * np.sin(self.angle) * np.cos(self.angle), -self.g/2*self.ejection_time**2 * np.sin(self.angle) * np.sin(self.angle), 0])
        waggon_eject_speed = np.array([self.g*self.ejection_time * np.sin(self.angle) * np.cos(self.angle), -self.g*self.ejection_time * np.sin(self.angle) * np.sin(self.angle), 0])
        # different time sections
        if (t < self.ejection_time):
            ball_pos = self.ax_c.c2p(*waggon_c) + 0.25/2*(np.cos(self.angle)*UP + np.sin(self.angle)*RIGHT)
        elif (t > self.ejection_time + 2*v0 / (self.g*np.cos(self.angle))):
            ball_pos = self.ax_c.c2p(*waggon_c) + 0.25/2*(np.cos(self.angle)*UP + np.sin(self.angle)*RIGHT)
        else:
            ball_pos = self.ax_c.c2p(*(waggon_eject_pos + waggon_eject_speed*(t-self.ejection_time) + ball_increment_c)) + 0.25/2*(np.cos(self.angle)*UP + np.sin(self.angle)*RIGHT)
        ball = Circle(radius = 0.125, color = PURE_RED, fill_color = PURE_RED, fill_opacity = 0.5).move_to(ball_pos)
        ball.position = ball_pos

        # ball vectors
        ball.gravitation = Line(start = ball_pos, end = ball_pos + 1.5*DOWN, color = PURE_RED).add_tip(tip_length = 0.15, tip_width = 0.15)
        ball.gravitation_descriptor = Tex(r"$\Vec{g}$", font_size = 32, color = PURE_RED).next_to(ball.gravitation, RIGHT).shift(0.15*LEFT)
        ball.perp = Line(start = ball_pos, end = ball_pos + 1.5*np.cos(self.angle)*(np.cos(self.angle)*DOWN - np.sin(self.angle)*RIGHT), color = PURE_RED).add_tip(tip_length = 0.15, tip_width = 0.15).set_opacity(0.5)
        ball.perp_descriptor = Tex(r"$\Vec{g}_\mathrm{y'}$", font_size = 32, color = PURE_RED).next_to(ball.perp, LEFT).shift(0.3*RIGHT).set_opacity(0.5)
        ball.line = Line(start = ball_pos, end = ball_pos + 1.5*np.sin(self.angle)*(np.sin(self.angle)*DOWN + np.cos(self.angle)*RIGHT), color = PURE_RED).add_tip(tip_length = 0.15, tip_width = 0.15).set_opacity(0.5)
        ball.line_descriptor = Tex(r"$\Vec{g}_\mathrm{x'}$", font_size = 32, color = PURE_RED).next_to(ball.line, RIGHT).shift(0.1*LEFT).set_opacity(0.5)
        return ball

In [6]:
%%manim -qh --fps 60 $video_scene


class ip_Scene(Scene):
    def construct(self):
        self.camera.background_color = WHITE

        center = np.array([-0.5, 0, 0])
        g = 25
        height = 3
        width = 8
        angle = np.arctan(height / width)

        # time controls
        starting_time = 0.25
        ejection_time = 0.25
        snapshot_time = 0.75
        end_time = 1.35

        # plane object
        inclined_plane = InclinedPlane(center, height = height, width = width, ejection_time = ejection_time, g = g)
        self.add(inclined_plane)

        # waggon and ball
        waggon = inclined_plane.get_waggon(starting_time)
        ball = inclined_plane.get_ball(starting_time)
        waggon.getter = inclined_plane.get_waggon
        ball.getter = inclined_plane.get_ball
        waggon.counter = 0
        ball.counter = 0
        self.add(waggon, ball)

        traj_group = VGroup()


        # coordinate labels
        ax_c_x_label = inclined_plane.ax_c_xlabel
        ax_c_y_label = inclined_plane.ax_c_ylabel
        ax_c_star_trans_x_label = inclined_plane.ax_c_star_xlabel
        ax_c_star_trans_y_label = inclined_plane.ax_c_star_ylabel
        ax_label_group = VGroup(ax_c_x_label, ax_c_y_label)
        self.add(ax_label_group)


        # updater for waggon and ball
        def physics_updater(object):
            object.counter += 1
            t = time.get_value()
            new_object = object.getter(t)
            if (object.counter % 20 == 0):
                traj_object = new_object.scale(0.5)
                traj_group.add(traj_object)
                self.add(traj_object)
            object.move_to(new_object.position)


        # animation
        time = ValueTracker(starting_time)
        waggon.add_updater(physics_updater)
        ball.add_updater(physics_updater)
        self.wait(1.5)
        self.play(time.animate.set_value(end_time), rate_func= linear, run_time = end_time * 3)
        waggon.remove_updater(physics_updater)
        ball.remove_updater(physics_updater)
        self.wait(3)
        
        # redefine waggon and ball
        waggon_renew = inclined_plane.get_waggon(starting_time)
        ball_renew = inclined_plane.get_ball(starting_time)

        self.play(FadeOut(traj_group), FadeTransform(waggon, waggon_renew), FadeTransform(ball, ball_renew), FadeIn(inclined_plane.ax_c_star), FadeIn(ax_c_star_trans_x_label), FadeIn(ax_c_star_trans_y_label),  run_time = 3)
        self.wait(1)


Manim Community v0.18.1

In [7]:
%%manim -qh --fps 60 $video2_scene


class ip2_Scene(Scene):
    def construct(self):
        self.camera.background_color = WHITE

        center = np.array([-0.5, 0, 0])
        g = 25
        height = 3
        width = 8
        angle = np.arctan(height / width)

        # time controls
        starting_time = 0.25
        ejection_time = 0.25
        snapshot_time = 0.75
        end_time = 1.35

        # plane object
        inclined_plane = InclinedPlane(center, height = height, width = width, ejection_time = ejection_time, g = g)
        self.add(inclined_plane, inclined_plane.ax_c_star)

        # waggon and ball
        waggon = inclined_plane.get_waggon(starting_time)
        ball = inclined_plane.get_ball(starting_time)
        waggon.getter = inclined_plane.get_waggon
        ball.getter = inclined_plane.get_ball
        waggon.counter = 0
        ball.counter = 0
        self.add(waggon, ball)

        traj_group = VGroup()


        # coordinate labels
        ax_c_x_label = inclined_plane.ax_c_xlabel
        ax_c_y_label = inclined_plane.ax_c_ylabel
        ax_c_star_trans_x_label = inclined_plane.ax_c_star_xlabel
        ax_c_star_trans_y_label = inclined_plane.ax_c_star_ylabel
        ax_label_group = VGroup(ax_c_x_label, ax_c_y_label, ax_c_star_trans_x_label, ax_c_star_trans_y_label)
        self.add(ax_label_group)


        # updater for waggon and ball
        def physics_updater(object):
            object.counter += 1
            t = time.get_value()
            new_object = object.getter(t)
            if (object.counter % 20 == 0):
                traj_object = new_object.scale(0.5)
                traj_group.add(traj_object)
                self.add(traj_object)
            object.move_to(new_object.position)


        # animation
        time = ValueTracker(starting_time)
        waggon.add_updater(physics_updater)
        ball.add_updater(physics_updater)
        self.wait(0.5)
        self.play(time.animate.set_value(snapshot_time), rate_func = linear, run_time = (snapshot_time-starting_time) * 3)

        # snapshot: vectors and descriptors
        waggon_snapshot = inclined_plane.get_waggon(time.get_value())
        waggon_snapshot_gravitation_vector = waggon_snapshot.gravitation
        waggon_snapshot_perp_vector = waggon_snapshot.perp
        waggon_snapshot_line_vector = waggon_snapshot.line

        ball_snapshot = inclined_plane.get_ball(time.get_value())
        ball_snapshot_gravitation_vector = ball_snapshot.gravitation
        ball_snapshot_perp_vector = ball_snapshot.perp
        ball_snapshot_line_vector = ball_snapshot.line

        waggon_snapshot_gravitation_descriptor = waggon_snapshot.gravitation_descriptor
        waggon_snapshot_perp_descriptor = waggon_snapshot.perp_descriptor
        waggon_snapshot_line_descriptor = waggon_snapshot.line_descriptor

        ball_snapshot_gravitation_descriptor = ball_snapshot.gravitation_descriptor
        ball_snapshot_perp_descriptor = ball_snapshot.perp_descriptor
        ball_snapshot_line_descriptor = ball_snapshot.line_descriptor
        descriptor_group = VGroup(
            waggon_snapshot_gravitation_descriptor, waggon_snapshot_perp_descriptor, waggon_snapshot_line_descriptor,
            ball_snapshot_gravitation_descriptor, ball_snapshot_perp_descriptor, ball_snapshot_line_descriptor)


        # remove updater before wait
        waggon.remove_updater(physics_updater)
        ball.remove_updater(physics_updater)
        # self.add(waggon_snapshot_gravitation_vector, waggon_snapshot_perp_vector, waggon_snapshot_line_vector)
        # self.add(ball_snapshot_gravitation_vector, ball_snapshot_perp_vector, ball_snapshot_line_vector)
        self.play(
            FadeIn(waggon_snapshot_gravitation_vector), FadeIn(waggon_snapshot_perp_vector), FadeIn(waggon_snapshot_line_vector), 
            FadeIn(ball_snapshot_gravitation_vector), FadeIn(ball_snapshot_perp_vector), FadeIn(ball_snapshot_line_vector), 
            FadeIn(waggon_snapshot_gravitation_descriptor), FadeIn(waggon_snapshot_perp_descriptor), FadeIn(waggon_snapshot_line_descriptor),
            FadeIn(ball_snapshot_gravitation_descriptor), FadeIn(ball_snapshot_perp_descriptor), FadeIn(ball_snapshot_line_descriptor),
            run_time = 1)
        self.wait(3)

        rotation_group = VGroup(
            inclined_plane.inclined_plane, inclined_plane.ax_c, inclined_plane.inclined_plane_fill, 
            waggon, ball, waggon_snapshot_gravitation_vector, waggon_snapshot_perp_vector, waggon_snapshot_line_vector, ball_snapshot_gravitation_vector, ball_snapshot_perp_vector, ball_snapshot_line_vector,
            *traj_group, inclined_plane.ax_c_star, *ax_label_group, *descriptor_group
            )

        self.play(Rotate(rotation_group, about_point = inclined_plane.ax_c.c2p(0, 0, 0), angle = angle), run_time = 1)

        # self.play()
        # copy opacity change objects
        inclined_plane_c_trans = inclined_plane.ax_c.copy().set_opacity(0.125)
        inclined_plane_c_star = inclined_plane.ax_c_star.copy().set_opacity(1)
        ax_c_trans_x_label = inclined_plane.ax_c_xlabel.copy().set_opacity(0.125)
        ax_c_trans_y_label = inclined_plane.ax_c_ylabel.copy().set_opacity(0.125)
        ax_c_star_x_label = inclined_plane.ax_c_star_xlabel.copy().set_opacity(1)
        ax_c_star_y_label = inclined_plane.ax_c_star_ylabel.copy().set_opacity(1)
        self.play(
            FadeTransform(inclined_plane.ax_c, inclined_plane_c_trans), FadeTransform(inclined_plane.ax_c_star, inclined_plane_c_star),
            FadeTransform(ax_c_x_label, ax_c_trans_x_label), FadeTransform(ax_c_y_label, ax_c_trans_y_label),
            FadeTransform(ax_c_star_trans_x_label, ax_c_star_x_label), FadeTransform(ax_c_star_trans_y_label, ax_c_star_y_label),
            Rotate(waggon_snapshot_gravitation_descriptor, angle = -angle),
            Rotate(waggon_snapshot_perp_descriptor, angle = -angle),
            Rotate(waggon_snapshot_line_descriptor, angle = -angle),
            Rotate(ball_snapshot_gravitation_descriptor, angle = -angle),
            Rotate(ball_snapshot_perp_descriptor, angle = -angle),
            Rotate(ball_snapshot_line_descriptor, angle = -angle),
            )
        self.wait(3)

        # finish
        self.play(
            FadeOut(waggon_snapshot_gravitation_vector), FadeOut(waggon_snapshot_perp_vector), FadeOut(waggon_snapshot_line_vector), 
            FadeOut(ball_snapshot_gravitation_vector), FadeOut(ball_snapshot_perp_vector), FadeOut(ball_snapshot_line_vector),
            FadeOut(waggon_snapshot_gravitation_descriptor), FadeOut(waggon_snapshot_perp_descriptor), FadeOut(waggon_snapshot_line_descriptor),
            FadeOut(ball_snapshot_gravitation_descriptor), FadeOut(ball_snapshot_perp_descriptor), FadeOut(ball_snapshot_line_descriptor),
            run_time = 1)
        waggon.add_updater(physics_updater)
        ball.add_updater(physics_updater)
        self.play(time.animate.set_value(end_time), rate_func= linear, run_time = (end_time-snapshot_time) * 3)
        waggon.remove_updater(physics_updater)
        ball.remove_updater(physics_updater)
        self.wait(3)

        # connectors
        connector_group = VGroup()
        for i in range(int(len(traj_group)/2)):
            start_connector = traj_group[2*i].get_center()
            end_connector = traj_group[2*i+1].get_center()
            dashed_line = DashedLine(start = start_connector, end = end_connector, color = BLACK, stroke_opacity = 0.5)
            connector_group.add(dashed_line)
        self.play(FadeIn(connector_group))
        self.wait(3)

Manim Community v0.18.1

In [8]:
# ffmpeg -f concat -i inclined_plane_merge_list.txt -c copy inclined_plane_FULL_1.mp4